In [1]:
import pandas as pd

In [15]:
df = pd.DataFrame([['BTC/ETH', 0.01], ['ETH/LTC', 0.05], ['LTC/BTC', 1000]])
df.columns = ['pair', 'rate']
df

,pair,rate
0,BTC/ETH,0.01
1,ETH/LTC,0.05
2,LTC/BTC,1000.00


In [16]:
df[['Numerator','Denumerator']] = df.pair.str.split("/",expand=True,)
df

,pair,rate,Numerator,Denumerator
0,BTC/ETH,0.01,BTC,ETH
1,ETH/LTC,0.05,ETH,LTC
2,LTC/BTC,1000.00,LTC,BTC


In [77]:
def triangularArbitrage(inputSet, firstTransaction, secondTransaction):
    # first transaction
    rate1 = df.rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)]
    output1 = inputSet[0]/rate1
    # second transaction
    rate2 = df.rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)]
    output2 = output1/rate2
    # third transaction
    rate3 = df.rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])]
    output3 = output2/rate3
    print(f'Obtained by {inputSet[1]} > {firstTransaction} > {secondTransaction} > {inputSet[1]}:', output3 - inputSet[0], inputSet[1])


inputSet = (1, 'BTC')
triangularArbitrage(inputSet, 'LTC', 'ETH')

Obtained by BTC > LTC > ETH > BTC: Series([], Name: rate, dtype: float64) BTC


In [80]:
transactionSets = [['ETH', 'LTC'], ['LTC', 'ETH']]
for i in transactionSets:
    print(i[0], i[1])
#     triangularArbitrage(inputSet, i[0], i[1])

ETH LTC
LTC ETH


### Create inverse rates

In [134]:
df = pd.read_csv('testData.csv')
df = (df
        .append(df)
        .drop(columns=['Unnamed: 0'])
        .reset_index()
     )

In [135]:
df[103:106]

,index,Currency,Pair,Volume,Rate,Numerator,Denumerator
103,0,Paxos Standard Token,PAX/USDT,2.508283e+09,1.026300,PAX,USDT
104,1,USD Coin,USDC/USDT,1.513212e+09,1.026290,USDC,USDT
105,2,BitMax Token,BTMX/USDT,1.500912e+07,0.112087,BTMX,USDT


In [136]:
for index in df[103:].index:
    df.loc[index, 'Rate'] = float(1/df.loc[index, 'Rate'])
    
    df.loc[index, 'Numerator1'] = df.loc[index, 'Denumerator']
    df.loc[index, 'Denumerator1'] = df.loc[index, 'Numerator']
   
    df.loc[index, 'Numerator'] = df.loc[index, 'Numerator1']
    df.loc[index, 'Denumerator'] = df.loc[index, 'Denumerator1']

In [137]:
df = df.drop(columns=['index', 'Numerator1', 'Denumerator1'])
df.head(3)

,Currency,Pair,Volume,Rate,Numerator,Denumerator
0,Paxos Standard Token,PAX/USDT,2.508283e+09,1.026300,PAX,USDT
1,USD Coin,USDC/USDT,1.513212e+09,1.026290,USDC,USDT
2,BitMax Token,BTMX/USDT,1.500912e+07,0.112087,BTMX,USDT


### Real data arbitrage

In [142]:
df[:103].Denumerator.unique()

array(['USDT', 'BTC', 'PAX', 'ETH', 'USDC'], dtype=object)

In [143]:
df[:103].Numerator.unique()

array(['PAX', 'USDC', 'BTMX', 'BTC', 'ETH', 'BNB', 'RNT', 'XTZ', 'CVNT',
       'XRP', 'HT', 'QCX', 'ANKR', 'FTM', 'EOS', 'ONT', 'ETZ', 'BCH',
       'LAMB', 'NEO', 'COVA', 'BTT', 'LTC', 'LTO', 'BAT', 'CHX', 'KCS',
       'AERGO', 'BOLT', 'IOST', 'ETC', 'XLM', 'TRX', 'DOS', 'ZIL',
       'BCHSV', 'ZRX', 'ZEC', 'FET', 'ELF', 'LBA', 'ONG', 'GAS'],
      dtype=object)

In [178]:
def triangularArbitrage(inputSet, firstTransaction, secondTransaction):
    # first transaction
    rate1 = df.Rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)]
    output1 = inputSet[0]/rate1
    output1 = (output1
                   .reset_index()
                   .drop(columns = ['index']))
    output1 = float(output1.loc[0])
    
    # second transaction
    rate2 = df.Rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)]
    output2 = output1/rate2
    output2 = (output2
                   .reset_index()
                   .drop(columns = ['index']))
    output2 = float(output2.loc[0])
    
    # third transaction
    rate3 = df.Rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])]
    output3 = output2/rate3
    output3 = (output3
                   .reset_index()
                   .drop(columns = ['index']))
    output3 = float(output3.loc[0])
    
    print(f'Obtained by {inputSet[1]} > {firstTransaction} > {secondTransaction} > {inputSet[1]}:', float(output3 - inputSet[0]), inputSet[1])

inputSet = (1, 'BTC')
triangularArbitrage(inputSet, 'LTC', 'ETH')

Obtained by BTC > LTC > ETH > BTC: 0.0007017640515150081 BTC
